# Fine-Tuning the CNN on Cancer and Subtype





In [1]:
import os
import sys
from pathlib import Path

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
#Add the parent directory to access ENV variables
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

#Import of necessary paths ( GDC data Path and Dataset folder)
from config import THYROID_PATH, MODEL_PATH, RESULTS_PATH

2025-04-15 10:12:27.405751: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-15 10:12:27.476939: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-15 10:12:27.477003: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-15 10:12:27.477014: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-15 10:12:27.488198: I tensorflow/core/platform/cpu_feature_g

In [2]:
FinalPath = Path(THYROID_PATH,'FinalData.npy')
npzfiles = np.load(FinalPath,allow_pickle=True)


#output_folder = Path(RESULTS_PATH,'UnfilteredCancer/')
#output_folder.mkdir(exist_ok=True) #Create output folder if it does not exist 

In [3]:
def get_uncompiled_model(reset_last_layer=False):
    
    model = keras.models.load_model(os.path.join(MODEL_PATH,'pan-cancer-leaky-relu'))
    if(reset_last_layer):
        output_follicolar= keras.layers.Dense(1, activation='sigmoid',name='output_follicolar')(model.layers[-2].output)
        model = keras.models.Model(inputs=model.input, outputs = [output_follicolar])
    return model

def get_compiled_model(metrics=None,reset_last_layer=False):
    
    if(metrics is None):
        metrics = [
              keras.metrics.TruePositives(name='tp'),
              keras.metrics.FalsePositives(name='fp'),
              keras.metrics.TrueNegatives(name='tn'),
              keras.metrics.FalseNegatives(name='fn'), 
              keras.metrics.BinaryAccuracy(name='accuracy'),
              keras.metrics.Precision(name='precision'),
              keras.metrics.Recall(name='recall'),
              keras.metrics.AUC(name='auc'),
              keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
        ]
        
    model = get_uncompiled_model(reset_last_layer)
    model.compile(loss='binary_crossentropy',optimizer=keras.optimizers.Adam(3e-5),metrics=metrics)
    
    return model

early_stopping_cb = tf.keras.callbacks.EarlyStopping(
    monitor='prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

def make_or_restore_model(checkpoint_dir = "ckpt", reset_last_layer=False):
    # Either restore the latest model, or create a fresh one
    # if there is no checkpoint available.
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    checkpoints = [checkpoint_dir + "/" + name for name in os.listdir(checkpoint_dir)]
    if checkpoints:
        latest_checkpoint = max(checkpoints, key=os.path.getctime)
        print("Restoring from", latest_checkpoint)
        return keras.models.load_model(latest_checkpoint)
    print("Creating a new model")
    return get_compiled_model(reset_last_layer=reset_last_layer)

In [4]:
#Retrain Base Model for thyroid cancer classification

In [5]:
X_train = np.nan_to_num(npzfiles['X_cancer_train'])
y_train = npzfiles['y_cancer_train']

model = make_or_restore_model('ckpt-thyroid-cancer')

cancer_history = model.fit(X_train,y_train, batch_size=8, epochs=30, callbacks=early_stopping_cb, validation_split=0.2, shuffle=True)

cancer_df = pd.DataFrame(cancer_history.history)
cancer_df.to_csv("fit_history/thyroid-cancer.csv")
model.save(os.path.join(MODEL_PATH,"thyroid-cancer"))

Creating a new model
Epoch 1/30
42/42 [==============================] - 241s 6s/step - loss: 3.0700 - tp: 264.0000 - fp: 2.0000 - tn: 28.0000 - fn: 35.0000 - accuracy: 0.8875 - precision: 0.9925 - recall: 0.8829 - auc: 0.9137 - prc: 0.9875 - val_loss: 1.1994 - val_tp: 67.0000 - val_fp: 4.0000 - val_tn: 10.0000 - val_fn: 2.0000 - val_accuracy: 0.9277 - val_precision: 0.9437 - val_recall: 0.9710 - val_auc: 0.9110 - val_prc: 0.9681
Epoch 2/30
42/42 [==============================] - 228s 5s/step - loss: 2.0774 - tp: 290.0000 - fp: 10.0000 - tn: 20.0000 - fn: 9.0000 - accuracy: 0.9422 - precision: 0.9667 - recall: 0.9699 - auc: 0.8172 - prc: 0.9653 - val_loss: 1.3311 - val_tp: 67.0000 - val_fp: 4.0000 - val_tn: 10.0000 - val_fn: 2.0000 - val_accuracy: 0.9277 - val_precision: 0.9437 - val_recall: 0.9710 - val_auc: 0.8478 - val_prc: 0.9428
Epoch 3/30
42/42 [==============================] - 226s 5s/step - loss: 1.2942 - tp: 287.0000 - fp: 9.0000 - tn: 21.0000 - fn: 12.0000 - accuracy: 0.936

INFO:tensorflow:Assets written to: /home/jovyan/NAS/Subtypes-detection-of-papillary-thyroid-cancer-from-methylation-assay-via-Deep-Neural-Network/Models/thyroid-cancer/assets


In [6]:
X_train = np.nan_to_num(npzfiles['X_subtype_train'])
y_train = npzfiles['y_subtype_train']

model = make_or_restore_model('ckpt-thyroid-subtype', reset_last_layer=True)

subtype_history = model.fit(X_train,y_train, batch_size=8, epochs=30, callbacks=early_stopping_cb, validation_split=0.2, shuffle=True)
subtype_df = pd.DataFrame(subtype_history.history)
subtype_df.to_csv(os.path.join('fit_history','thyroid-subtype.csv'))

model.save(os.path.join(MODEL_PATH,"thyroid-subtype"))

Creating a new model
Epoch 1/30
37/37 [==============================] - 212s 6s/step - loss: 0.6178 - tp: 9.0000 - fp: 22.0000 - tn: 208.0000 - fn: 55.0000 - accuracy: 0.7381 - precision: 0.2903 - recall: 0.1406 - auc: 0.6140 - prc: 0.2802 - val_loss: 0.5864 - val_tp: 15.0000 - val_fp: 16.0000 - val_tn: 41.0000 - val_fn: 2.0000 - val_accuracy: 0.7568 - val_precision: 0.4839 - val_recall: 0.8824 - val_auc: 0.8535 - val_prc: 0.6023
Epoch 2/30
37/37 [==============================] - 204s 6s/step - loss: 0.5426 - tp: 14.0000 - fp: 23.0000 - tn: 207.0000 - fn: 50.0000 - accuracy: 0.7517 - precision: 0.3784 - recall: 0.2188 - auc: 0.6926 - prc: 0.3654 - val_loss: 0.4373 - val_tp: 10.0000 - val_fp: 5.0000 - val_tn: 52.0000 - val_fn: 7.0000 - val_accuracy: 0.8378 - val_precision: 0.6667 - val_recall: 0.5882 - val_auc: 0.8488 - val_prc: 0.5851
Epoch 3/30
37/37 [==============================] - 202s 5s/step - loss: 0.3939 - tp: 23.0000 - fp: 9.0000 - tn: 221.0000 - fn: 41.0000 - accuracy: 0.8

INFO:tensorflow:Assets written to: /home/jovyan/NAS/Subtypes-detection-of-papillary-thyroid-cancer-from-methylation-assay-via-Deep-Neural-Network/Models/thyroid-subtype/assets
